In [82]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os


In [83]:
!pip install opencv-python-headless
!apt-get install -y unzip

Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\certifi-2024.12.14-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\charset_normalizer-3.4.1-py3.11-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\idna-3.10-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\p

In [5]:
!pip install labelImg
!labelImg


Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.8 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 12.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 6.9/6.9 MB 46.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.1 MB ? eta -:--:--
   -- ------------------------------------- 2.9/50.1 MB 12.9 MB/s eta 0:00:04
   --- ------------------------------------ 3.9/50.1 MB 9.8 MB/s eta 0:00:05
   ----- ---------------------------------- 6.6/50.1 MB 9.8 MB/s eta 0:00:05
   ------ --------------------------------- 8.4/50.1 MB 9.8 MB/s eta 0:00:05
   ------- -------------------------------- 9.7/50.1 MB 8.9 MB/s eta 0:00

DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\certifi-2024.12.14-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\charset_normalizer-3.4.1-py3.11-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\idna-3.10-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\p

In [91]:
import os
import json
from PIL import Image  # Чтобы считать реальные размеры изображений

# Путь к датасету
dataset_path = "dataset-resized"
# Файл, куда сохраним аннотации
output_json_path = os.path.join(dataset_path, "annotations.json")

# Получаем список подпапок (классы) в датасете
class_names = []
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        class_names.append(item)

# Сортируем для единообразия
class_names = sorted(class_names)

# Создаём список категорий в стиле COCO (0-based индекс)
categories = [
    {"id": idx, "name": class_name} 
    for idx, class_name in enumerate(class_names)
]

# Создаём структуру под COCO-аннотации
annotations_data = {
    "images": [],
    "annotations": [],
    "categories": categories
}

image_id = 0
annotation_id = 0

# Обходим все подпапки (каждая подпапка = класс)
for class_id, class_name in enumerate(class_names):
    class_folder = os.path.join(dataset_path, class_name)
    
    # Пропускаем, если это вдруг не папка
    if not os.path.isdir(class_folder):
        continue

    # Перебираем все файлы в папке
    for filename in os.listdir(class_folder):
        # Проверяем, что файл — изображение (JPEG, PNG и т.д.)
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_id += 1

            image_path = os.path.join(class_folder, filename)

            # Считываем реальный размер изображения
            with Image.open(image_path) as img:
                width, height = img.size
            
            # Добавляем запись об изображении
            annotations_data["images"].append({
                "id": image_id,
                "file_name": os.path.join(class_name, filename),  # Сохраним относительный путь
                "width": width,
                "height": height
            })

            # Добавляем одну аннотацию, покрывающую всё изображение
            annotation_id += 1
            annotations_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "bbox": [0, 0, width, height],  # Весь кадр
                "area": float(width * height),
                "iscrowd": 0
            })

print(f"📂 Найдено {image_id} изображений в {len(class_names)} классах: {class_names}")

# Сохраняем всё в JSON
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(annotations_data, f, indent=2, ensure_ascii=False)

print(f"✅ Файл `annotations.json` сохранён в {output_json_path}")


📂 Найдено 2527 изображений в 7 классах: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'yolo_labels']
✅ Файл `annotations.json` сохранён в dataset-resized\annotations.json


In [93]:
import os
import json
from PIL import Image

dataset_path = "dataset-resized"
output_json_path = os.path.join(dataset_path, "annotations.json")

class_names = []
for item in os.listdir(dataset_path):
    if os.path.isdir(os.path.join(dataset_path, item)):
        class_names.append(item)
class_names = sorted(class_names)

categories = [
    {"id": idx, "name": class_name}
    for idx, class_name in enumerate(class_names)
]

annotations_data = {
    "images": [],
    "annotations": [],
    "categories": categories
}

image_id = 0
annotation_id = 0

for class_id, class_name in enumerate(class_names):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue
    
    for filename in os.listdir(class_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_id += 1
            
            image_path = os.path.join(class_folder, filename)
            
            with Image.open(image_path) as img:
                width, height = img.size
            
            # Делаем относительный путь с прямым слэшем
            file_name = os.path.join(class_name, filename).replace("\\", "/")
            
            annotations_data["images"].append({
                "id": image_id,
                "file_name": file_name,  # теперь будет "plastic/plastic366.jpg"
                "width": width,
                "height": height
            })
            
            annotation_id += 1
            annotations_data["annotations"].append({
                "id": annotation_id,
                "image_id": image_id,
                "category_id": class_id,
                "bbox": [0, 0, width, height],
                "area": float(width * height),
                "iscrowd": 0
            })

print(f"📂 Найдено {image_id} изображений в {len(class_names)} классах: {class_names}")

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(annotations_data, f, indent=2, ensure_ascii=False)

print(f"✅ Файл `annotations.json` сохранён в {output_json_path}")


📂 Найдено 2527 изображений в 7 классах: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'yolo_labels']
✅ Файл `annotations.json` сохранён в dataset-resized\annotations.json


In [94]:
import os
from PIL import Image

# Путь к датасету, где каждая подпапка = класс (например, plastic, metal и т.д.)
dataset_path = "dataset-resized"
# Папка, в которую будут сохраняться YOLO-метки
output_labels_path = "dataset-resized/yolo_labels"

# Сканируем подпапки (классы)
class_names = []
for item in os.listdir(dataset_path):
    item_path = os.path.join(dataset_path, item)
    if os.path.isdir(item_path):
        class_names.append(item)

# Сортируем для устойчивого порядка
class_names = sorted(class_names)

print("Найдены классы:", class_names)

# Обходим все классы
for class_id, class_name in enumerate(class_names):
    class_folder = os.path.join(dataset_path, class_name)
    if not os.path.isdir(class_folder):
        continue
    
    # Создадим аналогичную подпапку в output_labels_path для txt-файлов
    output_class_folder = os.path.join(output_labels_path, class_name)
    os.makedirs(output_class_folder, exist_ok=True)
    
    # Перебираем все изображения в папке
    for filename in os.listdir(class_folder):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(class_folder, filename)
            
            # Считаем реальные размеры изображения
            with Image.open(image_path) as img:
                width, height = img.size
            
            # Для упрощённой схемы: bbox = весь кадр
            # YOLO-формат: (class_id, x_center_norm, y_center_norm, w_norm, h_norm)
            # где x_center_norm, w_norm, ... = координаты/размеры / (width или height)
            
            # Т.к. хотим "весь кадр", то:
            # x_center = 0.5, y_center = 0.5
            # w_norm = 1.0, h_norm = 1.0
            # (при условии что всё изображение целиком — один объект)
            x_center_norm = 0.5
            y_center_norm = 0.5
            w_norm = 1.0
            h_norm = 1.0
            
            # Формируем имя txt-файла (то же имя, только расширение .txt)
            txt_filename = os.path.splitext(filename)[0] + ".txt"
            txt_path = os.path.join(output_class_folder, txt_filename)
            
            # Записываем одну строку
            # class_id x_center_norm y_center_norm w_norm h_norm
            with open(txt_path, "w", encoding="utf-8") as f:
                f.write(f"{class_id} {x_center_norm} {y_center_norm} {w_norm} {h_norm}\n")

print("✅ Готово! YOLO-текстовые файлы созданы в папке:", output_labels_path)


Найдены классы: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash', 'yolo_labels']
✅ Готово! YOLO-текстовые файлы созданы в папке: dataset-resized/yolo_labels


In [42]:
import os

dataset_path = r"dataset-resized"

# Создаем папки, если их нет
folders = [
    "images/train", "images/val", "images/test",
    "labels/train", "labels/val", "labels/test",
    "yolo_labels"
]

for folder in folders:
    path = os.path.join(dataset_path, folder)
    os.makedirs(path, exist_ok=True)
    print(f"📂 Создана папка: {path}")

print("\n✅ Структура папок готова!")


📂 Создана папка: dataset-resized\images/train
📂 Создана папка: dataset-resized\images/val
📂 Создана папка: dataset-resized\images/test
📂 Создана папка: dataset-resized\labels/train
📂 Создана папка: dataset-resized\labels/val
📂 Создана папка: dataset-resized\labels/test
📂 Создана папка: dataset-resized\yolo_labels

✅ Структура папок готова!


In [43]:
import shutil
import random

# Пути
image_classes = ["cardboard", "glass", "metal", "paper", "plastic", "trash"]
images_path = os.path.join(dataset_path, "images")

# Проход по каждой категории и перемещение файлов
for category in image_classes:
    category_path = os.path.join(dataset_path, category)
    if not os.path.exists(category_path):
        continue  # Если папки нет, пропускаем

    images = os.listdir(category_path)
    random.shuffle(images)

    # Разбиваем данные на 80% train, 10% val, 10% test
    num_train = int(len(images) * 0.8)
    num_val = int(len(images) * 0.1)

    for i, img in enumerate(images):
        src = os.path.join(category_path, img)
        if i < num_train:
            dest = os.path.join(images_path, "train", img)
        elif i < num_train + num_val:
            dest = os.path.join(images_path, "val", img)
        else:
            dest = os.path.join(images_path, "test", img)

        shutil.move(src, dest)

    print(f"📂 Категория '{category}' обработана! ({len(images)} файлов распределено)")

print("\n✅ Все изображения перемещены!")


📂 Категория 'cardboard' обработана! (403 файлов распределено)
📂 Категория 'glass' обработана! (501 файлов распределено)
📂 Категория 'metal' обработана! (410 файлов распределено)
📂 Категория 'paper' обработана! (594 файлов распределено)
📂 Категория 'plastic' обработана! (482 файлов распределено)
📂 Категория 'trash' обработана! (137 файлов распределено)

✅ Все изображения перемещены!


In [44]:
dataset_yaml = os.path.join(dataset_path, "dataset.yaml")

yaml_content = """\
path: {}
train: images/train
val: images/val
test: images/test

names:
  0: paper
  1: trash
  2: metal
  3: glass
  4: plastic
  5: cardboard
""".format(dataset_path.replace("\\", "/"))

with open(dataset_yaml, "w") as f:
    f.write(yaml_content)

print(f"✅ Файл {dataset_yaml} создан!")


✅ Файл dataset-resized\dataset.yaml создан!


In [46]:
from ultralytics import YOLO
import os

# Пути к датасету
dataset_path = r"dataset-resized"
images_root = os.path.join(dataset_path, "images")
labels_root = os.path.join(dataset_path, "labels")

# Загружаем предобученную YOLOv8 для разметки
model = YOLO("yolov8n.pt")  # YOLOv8 Nano - легкая модель

# Размечаем все сплиты (train, val, test)
splits = ["train", "val", "test"]

for split in splits:
    images_path = os.path.join(images_root, split)
    labels_path = os.path.join(labels_root, split)

    os.makedirs(labels_path, exist_ok=True)  # Создаём папку, если её нет

    for img_name in os.listdir(images_path):
        if img_name.endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(images_path, img_name)
            label_file = os.path.join(labels_path, img_name.replace(".jpg", ".txt").replace(".png", ".txt").replace(".jpeg", ".txt"))

            # Запускаем YOLO для предсказания bounding boxes
            results = model(img_path)

            # Сохраняем аннотации в YOLO формате
            with open(label_file, "w") as f:
                for result in results:
                    for box in result.boxes.data:
                        class_id = int(box[5])  # ID класса
                        x_center, y_center, width, height = map(float, box[:4])
                        f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    print(f"✅ Аннотации для {split} сохранены в {labels_path}/")

print("🎯 Полная автоматическая разметка завершена!")



image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard1.jpg: 480x640 (no detections), 13.3ms
Speed: 3.0ms preprocess, 13.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard10.jpg: 480x640 (no detections), 12.8ms
Speed: 0.0ms preprocess, 12.8ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard102.jpg: 480x640 (no detections), 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard103.jpg: 480x640 (no detections), 6.1ms
Speed: 2.0ms preprocess, 6.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 

In [47]:
import os

num_labels = len(os.listdir(labels_path))
print(f"📝 Создано {num_labels} аннотаций для {len(os.listdir(images_path))} изображений!")


📝 Создано 257 аннотаций для 257 изображений!


In [48]:
import os
import shutil

dataset_path = r"dataset-resized"
labels_train_path = os.path.join(dataset_path, "labels/train")
yolo_labels_path = os.path.join(dataset_path, "yolo_labels")

# Создаем папку yolo_labels, если её нет
os.makedirs(yolo_labels_path, exist_ok=True)

# Копируем все .txt файлы из labels/train в yolo_labels
for file_name in os.listdir(labels_train_path):
    if file_name.endswith(".txt"):
        src = os.path.join(labels_train_path, file_name)
        dst = os.path.join(yolo_labels_path, file_name)
        shutil.copy(src, dst)

print("✅ Все аннотации скопированы в yolo_labels/!")


✅ Все аннотации скопированы в yolo_labels/!


In [49]:
num_files = len(os.listdir(yolo_labels_path))
print(f"📂 В yolo_labels/ теперь {num_files} файлов-аннотаций!")


📂 В yolo_labels/ теперь 2019 файлов-аннотаций!


In [50]:
num_train = len(os.listdir(labels_train_path))
num_val = len(os.listdir(labels_val_path))
num_test = len(os.listdir(labels_test_path))

print(f"📂 Train: {num_train} аннотаций")
print(f"📂 Val: {num_val} аннотаций")
print(f"📂 Test: {num_test} аннотаций")


📂 Train: 2019 аннотаций
📂 Val: 251 аннотаций
📂 Test: 257 аннотаций


In [51]:
import os

dataset_path = r"C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized"
images_train = os.path.join(dataset_path, "images/train")
images_val = os.path.join(dataset_path, "images/val")
images_test = os.path.join(dataset_path, "images/test")

labels_train = os.path.join(dataset_path, "labels/train")
labels_val = os.path.join(dataset_path, "labels/val")
labels_test = os.path.join(dataset_path, "labels/test")

def check_annotations(images_folder, labels_folder):
    images = {os.path.splitext(f)[0] for f in os.listdir(images_folder) if f.endswith((".jpg", ".png", ".jpeg"))}
    labels = {os.path.splitext(f)[0] for f in os.listdir(labels_folder) if f.endswith(".txt")}

    missing_labels = images - labels
    missing_images = labels - images

    return missing_labels, missing_images

# Проверяем train, val, test
missing_train_labels, missing_train_images = check_annotations(images_train, labels_train)
missing_val_labels, missing_val_images = check_annotations(images_val, labels_val)
missing_test_labels, missing_test_images = check_annotations(images_test, labels_test)

# Выводим результаты
print(f"📂 Train: {len(missing_train_labels)} изображений без аннотаций, {len(missing_train_images)} аннотаций без изображений")
print(f"📂 Val: {len(missing_val_labels)} изображений без аннотаций, {len(missing_val_images)} аннотаций без изображений")
print(f"📂 Test: {len(missing_test_labels)} изображений без аннотаций, {len(missing_test_images)} аннотаций без изображений")

# Если файлы отсутствуют, выводим их имена
if missing_train_labels:
    print("🚨 В Train есть изображения без аннотаций:", missing_train_labels)
if missing_train_images:
    print("🚨 В Train есть аннотации без изображений:", missing_train_images)

if missing_val_labels:
    print("🚨 В Val есть изображения без аннотаций:", missing_val_labels)
if missing_val_images:
    print("🚨 В Val есть аннотации без изображений:", missing_val_images)

if missing_test_labels:
    print("🚨 В Test есть изображения без аннотаций:", missing_test_labels)
if missing_test_images:
    print("🚨 В Test есть аннотации без изображений:", missing_test_images)


📂 Train: 0 изображений без аннотаций, 0 аннотаций без изображений
📂 Val: 0 изображений без аннотаций, 0 аннотаций без изображений
📂 Test: 0 изображений без аннотаций, 0 аннотаций без изображений


In [52]:
import os
import json

# Пути к папкам
dataset_path = r"C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized"
images_path = os.path.join(dataset_path, "images")
labels_path = os.path.join(dataset_path, "labels")

# Классы (из dataset.yaml)
classes = ["paper", "trash", "metal", "glass", "plastic", "cardboard"]

# COCO формат
coco = {
    "images": [],
    "annotations": [],
    "categories": [{"id": i, "name": name} for i, name in enumerate(classes)]
}

# ID-счётчики
image_id = 0
annotation_id = 0

# Сканируем изображения
for split in ["train", "val", "test"]:
    split_path = os.path.join(images_path, split)
    
    for filename in os.listdir(split_path):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            image_id += 1
            file_path = os.path.join(split_path, filename)

            # Добавляем изображение в COCO
            coco["images"].append({
                "id": image_id,
                "file_name": filename,
                "width": 640,  # Нужно указать реальные размеры!
                "height": 640
            })

            # Ищем аннотации
            label_file = os.path.join(labels_path, split, filename.replace(".jpg", ".txt").replace(".png", ".txt"))
            
            if os.path.exists(label_file):
                with open(label_file, "r") as f:
                    for line in f.readlines():
                        parts = line.strip().split()
                        if len(parts) == 5:
                            category_id = int(parts[0])
                            x_center, y_center, width, height = map(float, parts[1:])

                            # Преобразуем в COCO bbox (x, y, w, h)
                            x = (x_center - width / 2) * 640
                            y = (y_center - height / 2) * 640
                            w = width * 640
                            h = height * 640

                            annotation_id += 1
                            coco["annotations"].append({
                                "id": annotation_id,
                                "image_id": image_id,
                                "category_id": category_id,
                                "bbox": [x, y, w, h],
                                "area": w * h,
                                "iscrowd": 0
                            })

# Сохраняем COCO-аннотации
annotations_path = os.path.join(dataset_path, "annotations.json")
with open(annotations_path, "w") as f:
    json.dump(coco, f, indent=4)

print(f"✅ Файл `annotations.json` создан! ({len(coco['annotations'])} аннотаций)")


✅ Файл `annotations.json` создан! (1617 аннотаций)


In [53]:
import json

# Путь к файлу аннотаций
annotations_path = r"C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\annotations.json"

# Загружаем JSON
with open(annotations_path, "r") as f:
    data = json.load(f)

# Выводим первые 5 аннотаций
print(json.dumps(data["annotations"][:5], indent=4))


[
    {
        "id": 1,
        "image_id": 5,
        "category_id": 72,
        "bbox": [
            -65824.52392578125,
            16229.21875,
            203914.609375,
            245602.32421875
        ],
        "area": 50081902004.65851,
        "iscrowd": 0
    },
    {
        "id": 2,
        "image_id": 7,
        "category_id": 0,
        "bbox": [
            -25267.31201171875,
            65696.865234375,
            50534.6240234375,
            238970.44921875
        ],
        "area": 12076281803.981495,
        "iscrowd": 0
    },
    {
        "id": 3,
        "image_id": 17,
        "category_id": 74,
        "bbox": [
            45909.619140625,
            45552.8564453125,
            136581.42578125,
            136703.798828125
        ],
        "area": 18671199753.658485,
        "iscrowd": 0
    },
    {
        "id": 4,
        "image_id": 18,
        "category_id": 0,
        "bbox": [
            -36253.2666015625,
            -10167.310791015625

In [57]:
import json
import os

# Пути
dataset_path = r"C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized"
annotations_path = os.path.join(dataset_path, "annotations.json")
labels_path = os.path.join(dataset_path, "labels")

# Загружаем COCO-аннотации
with open(annotations_path, "r") as f:
    data = json.load(f)

# Убедимся, что labels/ папки существуют
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(labels_path, split), exist_ok=True)

# Создаём YOLO аннотации
for annotation in data["annotations"]:
    image_id = annotation["image_id"]
    category_id = annotation["category_id"]
    bbox = annotation["bbox"]

    # YOLO формат: class x_center y_center width height (нормализованные)
    x, y, w, h = bbox
    image_width = 640  # !!! Укажите реальные размеры изображений
    image_height = 640  # !!! Укажите реальные размеры изображений

    x_center = (x + w / 2) / image_width
    y_center = (y + h / 2) / image_height
    width = w / image_width
    height = h / image_height

    # Определяем, в каком сплите находится изображение
    split = "train" if image_id <= 2019 else "val" if image_id <= 2019 + 251 else "test"

    # Сохраняем в YOLO-формате
    label_path = os.path.join(labels_path, split, f"{image_id}.txt")
    with open(label_path, "a") as f:
        f.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

print("✅ COCO-аннотации успешно сконвертированы в YOLO!")


✅ COCO-аннотации успешно сконвертированы в YOLO!


In [59]:
import os

labels_train_path = r"dataset-resized/labels/train"
txt_files = sorted([f for f in os.listdir(labels_train_path) if f.endswith(".txt")])

# Проверяем количество файлов
print(f"📂 Найдено {len(txt_files)} аннотаций в labels/train/")

expected_files = set(f"{i}.txt" for i in range(1, 2020))  # Должны быть от 1.txt до 2019.txt
actual_files = set(txt_files)

# Проверяем отсутствующие файлы
missing_files = expected_files - actual_files
extra_files = actual_files - expected_files

if missing_files:
    print(f"🚨 Недостающие файлы: {missing_files}")
else:
    print("✅ Все файлы от 1.txt до 2019.txt созданы правильно!")

if extra_files:
    print(f"⚠️ Лишние файлы: {extra_files}")


📂 Найдено 967 аннотаций в labels/train/
🚨 Недостающие файлы: {'1306.txt', '1001.txt', '164.txt', '1783.txt', '314.txt', '1201.txt', '1330.txt', '1340.txt', '961.txt', '338.txt', '997.txt', '436.txt', '766.txt', '814.txt', '297.txt', '1416.txt', '262.txt', '2005.txt', '917.txt', '605.txt', '31.txt', '1479.txt', '254.txt', '1061.txt', '111.txt', '1415.txt', '553.txt', '1487.txt', '862.txt', '163.txt', '1323.txt', '1695.txt', '1843.txt', '1799.txt', '304.txt', '573.txt', '51.txt', '1573.txt', '1750.txt', '207.txt', '440.txt', '1315.txt', '1075.txt', '266.txt', '1688.txt', '596.txt', '1171.txt', '1860.txt', '1244.txt', '995.txt', '741.txt', '1209.txt', '952.txt', '898.txt', '151.txt', '1490.txt', '478.txt', '225.txt', '1518.txt', '1612.txt', '113.txt', '1102.txt', '155.txt', '1073.txt', '1901.txt', '1322.txt', '610.txt', '1445.txt', '1358.txt', '658.txt', '290.txt', '773.txt', '396.txt', '1443.txt', '1455.txt', '1627.txt', '889.txt', '23.txt', '249.txt', '1569.txt', '259.txt', '828.txt', '

In [60]:
import os

# Пути к папкам labels/val и labels/test
labels_val_path = r"dataset-resized/labels/val"
labels_test_path = r"dataset-resized/labels/test"

# Функция для удаления всех файлов в папке
def clear_labels_folder(path):
    for file in os.listdir(path):
        file_path = os.path.join(path, file)
        if file.endswith(".txt"):
            os.remove(file_path)
    print(f"✅ Очистили папку {path}")

# Очищаем обе папки
clear_labels_folder(labels_val_path)
clear_labels_folder(labels_test_path)


✅ Очистили папку dataset-resized/labels/val
✅ Очистили папку dataset-resized/labels/test


In [73]:
import os
import glob
import json
from ultralytics import YOLO
from PIL import Image

# Пути в твоем проекте
dataset_path = "dataset-resized"
output_json_path = os.path.join(dataset_path, "annotations.json")  # Итоговый JSON
model_path = "yolov8n.pt"  # Предобученная YOLO-модель

# Папки с изображениями
splits = ["train", "val", "test"]  # Разделы датасета

def create_auto_annotations(dataset_path: str, output_json_path: str, model_path: str):
    """
    Автоматически создаёт аннотации COCO-формата для `train`, `val`, `test`.
    
    Параметры:
        dataset_path (str): Путь к папке с подкаталогами `train/val/test`.
        output_json_path (str): Путь для сохранения `annotations.json`.
        model_path (str): Путь к весам YOLO.
    """
    model = YOLO(model_path)

    # Формат COCO JSON
    annotations_data = {
        "images": [],
        "annotations": [],
        "categories": []
    }

    # Добавляем классы из YOLO в "categories"
    categories = {idx: name for idx, name in model.names.items()}
    for idx, name in categories.items():
        annotations_data["categories"].append({"id": idx, "name": name})

    annotation_id = 0
    image_id = 0

    # Проходим по `train`, `val`, `test`
    for split in splits:
        images_folder = os.path.join(dataset_path, f"images/{split}")
        image_paths = glob.glob(os.path.join(images_folder, "*.jpg"))

        for img_path in image_paths:
            filename = os.path.basename(img_path)

            # Получаем размеры изображения
            img = Image.open(img_path)
            width, height = img.size

            image_id += 1  # Новый `image_id` для каждого изображения

            # Добавляем изображение в JSON
            annotations_data["images"].append({
                "id": image_id,
                "file_name": filename,
                "width": width,
                "height": height,
                "split": split  # Указываем, к какому набору принадлежит изображение
            })

            # Получаем предсказания YOLO
            results = model(img_path)

            # Обрабатываем bounding boxes
            for result in results:
                for box in result.boxes:
                    cls_id = int(box.cls[0].item())  # ID класса
                    x_min, y_min, x_max, y_max = box.xyxy[0].tolist()

                    width_bbox = x_max - x_min
                    height_bbox = y_max - y_min

                    annotation_id += 1
                    annotations_data["annotations"].append({
                        "id": annotation_id,
                        "image_id": image_id,
                        "category_id": cls_id,
                        "bbox": [x_min, y_min, width_bbox, height_bbox],
                        "area": width_bbox * height_bbox,
                        "iscrowd": 0
                    })

            print(f"✅ {split.upper()} - Аннотация создана для {filename}")

    # Сохраняем JSON
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(annotations_data, f, indent=2, ensure_ascii=False)

    print(f"🎯 Аннотации сохранены в {output_json_path}")

if __name__ == "__main__":
    create_auto_annotations(dataset_path, output_json_path, model_path)



image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard1.jpg: 480x640 (no detections), 9.2ms
Speed: 2.0ms preprocess, 9.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
✅ TRAIN - Аннотация создана для cardboard1.jpg

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard10.jpg: 480x640 (no detections), 7.8ms
Speed: 2.0ms preprocess, 7.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
✅ TRAIN - Аннотация создана для cardboard10.jpg

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard102.jpg: 480x640 (no detections), 10.3ms
Speed: 1.0ms preprocess, 10.3ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
✅ TRAIN - Аннотация создана для cardboard102.jpg

image 1/1 c:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\images\train\cardboard103.j

In [81]:
import os
import json

# Пути в твоём проекте
json_path = "dataset-resized/annotations.json"  # Файл аннотаций
output_folder = "dataset-resized/yolo_labels"  # Папка для YOLO-аннотаций

# Создаём папку, если её нет
os.makedirs(output_folder, exist_ok=True)

# Загружаем JSON-аннотации
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Проверка структуры JSON
if "images" not in data or "annotations" not in data or "categories" not in data:
    raise ValueError("❌ Ошибка: Неверная структура JSON!")

# Создаём словарь {image_id: (filename, width, height)}
image_info = {img["id"]: (img["file_name"], img["width"], img["height"]) for img in data["images"]}

# Создаём словарь категорий {category_id: class_id}
category_mapping = {cat["id"]: idx for idx, cat in enumerate(data["categories"])}

# Проверяем, не выходят ли классы за границы
max_class_id = max(category_mapping.values())  # Максимальный class_id

# Группируем аннотации по изображению
annotations_by_image = {}
for ann in data["annotations"]:
    img_id = ann["image_id"]
    if img_id not in annotations_by_image:
        annotations_by_image[img_id] = []
    annotations_by_image[img_id].append(ann)

# Обрабатываем изображения и создаём YOLO-аннотации
missing_annotations = 0
for img_id, (filename, width, height) in image_info.items():
    output_txt_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.txt")

    # Проверяем, есть ли аннотации
    annotations = annotations_by_image.get(img_id, [])
    if not annotations:
        missing_annotations += 1
        continue  # Пропускаем изображения без аннотаций

    with open(output_txt_path, "w") as f:
        for ann in annotations:
            cat_id = ann["category_id"]

            # Проверяем, есть ли этот category_id в category_mapping
            if cat_id not in category_mapping:
                print(f"⚠️ Ошибка: category_id {cat_id} не найден в categories.json")
                continue

            class_id = category_mapping[cat_id]  # YOLO class_id

            # Проверяем, не выходит ли class_id за границы
            if class_id > max_class_id:
                print(f"❌ Ошибка: class_id {class_id} слишком большой для {filename} (макс {max_class_id})")
                continue

            x, y, w, h = ann["bbox"]

            # YOLO формула: x_center = (x + w/2) / width
            x_center = (x + w / 2) / width
            y_center = (y + h / 2) / height
            norm_w = w / width
            norm_h = h / height

            # Записываем в файл в формате YOLO
            f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {norm_w:.6f} {norm_h:.6f}\n")

    print(f"✅ Аннотация сохранена: {output_txt_path}")

print(f"🎯 Все аннотации успешно преобразованы в YOLO-формат!")
print(f"📂 Всего изображений: {len(image_info)}")
print(f"📂 Найдено аннотаций: {len(annotations_by_image)}")
print(f"⚠️ Пропущено {missing_annotations} изображений без аннотаций.")


✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard104.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard106.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard118.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard119.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard121.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard122.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard131.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard132.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard134.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard138.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard14.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard140.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard143.txt
✅ Аннотация сохранена: dataset-resized/yolo_labels\cardboard145.txt
✅ Аннотация сохранена: dataset-resized/yolo_label

In [63]:
import os
import shutil

# Пути
old_labels_path = r"dataset-resized/labels/train"  # Где сейчас аннотации
yolo_labels_path = r"dataset-resized/yolo_labels"  # Куда переместить

# Создаем папку для YOLO-аннотаций (если её нет)
os.makedirs(yolo_labels_path, exist_ok=True)

# Переносим все .txt файлы
for file_name in os.listdir(old_labels_path):
    if file_name.endswith(".txt"):
        old_file = os.path.join(old_labels_path, file_name)
        new_file = os.path.join(yolo_labels_path, file_name)
        shutil.move(old_file, new_file)

print(f"✅ Аннотации перемещены в {yolo_labels_path}")


✅ Аннотации перемещены в dataset-resized/yolo_labels


In [65]:
# Проверяем количество файлов в папке
num_labels = len(os.listdir(yolo_labels_path))
print(f"📂 Найдено {num_labels} аннотаций в yolo_labels/")


📂 Найдено 2019 аннотаций в yolo_labels/
